# Convert — Run a simple conversion of a PDB to the QDXF format

This notebook shows how to perform a simple conversion of PDBs to the QDXF format

# 0) Complete example
See the [sample notebook](/Quickstarts/convert-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import os
from glob import glob
from pathlib import Path

from pdbtools import pdb_fetch

import rush

## 1.1) Configuration

In [ ]:
# Define our project information
DESCRIPTION = "rush-py batch notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "convert"]
WORK_DIR = Path.home() / "qdx" / "convert"

# Set our inputs
PROTEIN_PDB_FOLDER_PATH = WORK_DIR / "proteins"

In [ ]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)
YOUR_TOKEN = os.getenv("RUSH_TOKEN")

2024-04-09 14:00:14,166 - rush - INFO - Not restoring by default via default


In [ ]:
os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

## 1.2) Build your client

Initialize our rush client and fetch available module paths.

In [ ]:
# Get our client, for calling modules and using the rush API
os.environ["RUSH_TOKEN"] = YOUR_TOKEN

client = rush.build_blocking_provider_with_functions(
    workspace=WORK_DIR,
    batch_tags=TAGS,
)

2024-04-09 14:00:14,390 - rush - INFO - Not restoring by default via default


# 2) Fetch PDBS

In [ ]:
pdb_ids = ["1B39", "1GIH"]
for pdb_id in pdb_ids:
    complex = list(pdb_fetch.fetch_structure(pdb_id))
    with open(PROTEIN_PDB_FOLDER_PATH / f"{pdb_id}.pdb", "w") as f:
        for line in complex:
            f.write(line)

# 3) Convert each PDB

In [ ]:
for pdb in glob(str(PROTEIN_PDB_FOLDER_PATH / "*.pdb")):
    (conformer,) = client.convert(
        "PDB", Path(pdb), tags=[Path(pdb).name], target="NIX"
    )
    print(conformer.download(overwrite=True).open().read()[0:50], "...")

2024-04-09 14:00:17,928 - rush - INFO - Argument 0dffd14c-f94c-41be-9623-20f627681919 is now ModuleInstanceStatus.RESOLVING
2024-04-09 14:00:19,030 - rush - INFO - Argument 0dffd14c-f94c-41be-9623-20f627681919 is now ModuleInstanceStatus.ADMITTED
2024-04-09 14:00:20,162 - rush - INFO - Argument 0dffd14c-f94c-41be-9623-20f627681919 is now ModuleInstanceStatus.RUNNING
2024-04-09 14:00:21,283 - rush - INFO - Argument 0dffd14c-f94c-41be-9623-20f627681919 is now ModuleInstanceStatus.AWAITING_UPLOAD
[{"topology": {"version": null, "symbols": ["C", " ...
2024-04-09 14:00:26,055 - rush - INFO - Argument e8cfcff6-424a-4698-b0c5-e27826ba2f12 is now ModuleInstanceStatus.RESOLVING
2024-04-09 14:00:27,178 - rush - INFO - Argument e8cfcff6-424a-4698-b0c5-e27826ba2f12 is now ModuleInstanceStatus.ADMITTED
2024-04-09 14:00:28,340 - rush - INFO - Argument e8cfcff6-424a-4698-b0c5-e27826ba2f12 is now ModuleInstanceStatus.RUNNING
2024-04-09 14:00:29,450 - rush - INFO - Argument e8cfcff6-424a-4698-b0c5-e278

In [ ]:
client.status()

{'4332ca04-d445-44a1-8c97-01a5eb6ff6bb': (<ModuleInstanceStatus.COMPLETED: 'COMPLETED'>,
  'convert',
  1),
 '09fd6693-3d81-4616-92a0-1086d9fbd139': (<ModuleInstanceStatus.COMPLETED: 'COMPLETED'>,
  'convert',
  1)}